# Exploratory Data Analysis
In this notebook, we will look at data on the movies we have available to include in our modeling. Not all of this information will be used in modeling, but it will let us see trends and features of our data that may influence how our recommender system is set up or interpretted.